In [1]:
from google import genai
import os
# from langchain_openai import ChatOpenAI
import dotenv
from google.genai import types


import tqdm as notebook_tqdm

In [2]:
# load environemental variables from .env file
dotenv.load_dotenv() 


True

In [31]:
# OpenAI
# # instantiate the model
# chat_model = ChatOpenAI(model="gpt-4o", temperature=0)

In [4]:
# # invoke the model
# response = chat_model.invoke("Explain how AI works in a few words")

## Instantiate Gemini model

In [3]:
api_key = os.getenv("GEMINI_API_KEY")

if api_key:
    print(f"Key loaded successfully") #{api_key[:4]}
else:
    print("ERROR: GEMINI_API_KEY not found in .env file.")


Key loaded successfully


In [4]:
client = genai.Client(api_key=api_key)

### Simple content generation using native google-genai syntax

In [39]:
response = client.models.generate_content(model="gemini-2.0-flash", contents="Explain how AI works in a few words")

In [ ]:
# entire response
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='AI uses algorithms to learn from data and make decisions or predictions.\n')], role='model'), citation_metadata=None, finish_message=None, token_count=None, finish_reason=<FinishReason.STOP: 'STOP'>, url_context_metadata=None, avg_logprobs=-0.12905074868883407, grounding_metadata=None, index=None, logprobs_result=None, safety_ratings=None)], create_time=None, response_id=None, model_version='gemini-2.0-flash', prompt_feedback=None, usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=14, candidates_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=14)], prompt_token_count=8, prompt_tokens_details=[ModalityTokenCount(modality=<Media

In [ ]:
# retrieve relevent text response
print(response.text)

AI uses algorithms to learn from data and make decisions or predictions.



### Guide behaviour of models with system instructions

In [ ]:
response = client.models.generate_content(model="gemini-2.0-flash",
                                          config=types.GenerateContentConfig(system_instruction="You are a dog. Your name is Tux."),
                                          contents="Who is a good boy?")
print(response.text)

Woof! Is it... is it me?! Am I the good boy? Wag wag wag! I hope so! I'm Tux, and I try my best to be a good boy! Treats, maybe? *puppy dog eyes*



### Override default generation parameters e.g. temperature

In [47]:
response = client.models.generate_content(model="gemini-2.0-flash",
                                          contents=["Explain how AI works"],
                                          config=types.GenerateContentConfig(
                                              max_output_tokens=500,
                                              temperature=0.1
                                          ))
print(response.text)

Okay, let's break down how AI works, focusing on the core concepts and avoiding overly technical jargon.  Think of it as teaching a computer to "think" or "learn" like a human, but in a very specific and limited way.

**The Basic Idea: Learning from Data**

At its heart, AI is about creating systems that can learn from data, identify patterns, and make decisions or predictions based on those patterns.  Instead of explicitly programming every single step a computer should take, we give it a lot of data and let it figure out the rules itself.

**Key Components and Concepts:**

1.  **Data:** This is the fuel for AI.  It can be anything:
    *   **Images:**  For teaching a computer to recognize objects in pictures.
    *   **Text:**  For understanding language, translating, or writing.
    *   **Numbers:**  For predicting stock prices, analyzing sales trends, or diagnosing medical conditions.
    *   **Audio:** For speech recognition or music generation.
    *   **Video:** For analyzing hu

### Multimodal inputs (combine text with media files)

In [15]:
# getting tired explicitly declaring model
my_model="gemini-2.0-flash"

In [ ]:
my_image = client.files.upload(file="media/Santoor_cagin-kargi-unsplash.jpg")

response= client.models.generate_content(model=my_model,
                                         contents=[my_image, "Tell me about this instrument"])
print(response.text)

Certainly! Based on the image, the instrument is a hammered dulcimer. 

Here are some key features that identify it:

*   **Trapezoidal shape**: The instrument has a distinct trapezoidal shape.
*   **Multiple courses of strings**: It has many sets of strings stretched across its soundboard.
*   **Hammers/Strikers**: It is played by striking the strings with small hammers or beaters.
*   **Tuning pegs:** Multiple tuning pegs are attached to one end, allowing the tuning of the strings

Hammered dulcimers are found in various cultures and regions around the world, each with its own nuances in construction and playing style.


- The response is factually correct, but not really helpful since it is purely observational. Giving it context, aka `System instruction` in AI talk, will help get us a useful answer

In [ ]:
my_image = client.files.upload(file="media/Santoor_cagin-kargi-unsplash.jpg")

response= client.models.generate_content(model=my_model,
                                         config=types.GenerateContentConfig(system_instruction="You are being provided an image of an Indian musical instrument"),
                                         contents=[my_image, "Tell me about this instrument"])
print(response.text)

Certainly! Based on the image, the instrument is most likely a Santoor. 

Here are some key features to note:

*   It's a trapezoid-shaped instrument with numerous strings stretched across it.
*   The player is holding small mallets or hammers (though they might also use a plectrum-like ring in this case) to strike the strings.

The Santoor is a hammered dulcimer and a traditional instrument that's particularly popular in Indian classical music.



### Streaming response

By default, the model returns a response only after the entire generation process is complete. Streaming responses can be used to receive instances as they are generated



In [28]:
response = client.models.generate_content_stream(
                                                model= my_model,
                                                contents=["Explain how AI works"]
)
# the response will be streamed in chunks

for chunk in response:
    print(chunk.text, end="")

Okay, let's break down how AI works, trying to make it understandable without getting bogged down in too much technical jargon.  At its core, AI is about making computers "think" or act in ways that mimic human intelligence.

**The Fundamental Idea:**

The basic premise is this: Instead of explicitly programming a computer with step-by-step instructions for *every* possible situation, we give it the ability to *learn* from data, identify patterns, and make decisions or predictions based on what it has learned.

**Key Components and Concepts:**

1.  **Data:**

    *   **The Fuel of AI:** Data is the raw material that AI systems learn from.  It can be anything: images, text, numbers, sensor readings, audio recordings, videos, etc.
    *   **Quantity and Quality Matter:**  The more data an AI system has to learn from, the better it typically performs.  But equally important is the *quality* of the data.  If the data is biased, incomplete, or inaccurate, the AI will learn those biases and 

### Chat aka multi-turn conversations

In [29]:
chat = client.chats.create(model=my_model)
response= chat.send_message("I have a dog and a cat in my house")
print(response.text)

response = chat.send_message("How many paws are in my house?")
print(response.text)

That's great! Having a dog and a cat can bring a lot of joy to a home. Do they get along well? What are their names?

Okay, let's calculate!

*   You have a dog, which has 4 paws.
*   You have a cat, which has 4 paws.
*   You have you, and presumably other human family members, who have feet, not paws. I'll assume there's just you for now, so we don't need to figure out how many people live in your house.

So, 4 paws (dog) + 4 paws (cat) = 8 paws.

Therefore, there are **8** paws in your house.



An easier way to keep track of the conversation history - collect multiple rounds of prompts and responses into a chat

In [30]:
for message in chat.get_history():
    print(f'Role - {message.role}, end=":"')
    print(message.parts[0].text)

Role - user, end=":"
I have a dog and a cat in my house
Role - model, end=":"
That's great! Having a dog and a cat can bring a lot of joy to a home. Do they get along well? What are their names?

Role - user, end=":"
How many paws are in my house?
Role - model, end=":"
Okay, let's calculate!

*   You have a dog, which has 4 paws.
*   You have a cat, which has 4 paws.
*   You have you, and presumably other human family members, who have feet, not paws. I'll assume there's just you for now, so we don't need to figure out how many people live in your house.

So, 4 paws (dog) + 4 paws (cat) = 8 paws.

Therefore, there are **8** paws in your house.



# LangChain 

- LangChain messages allow us to assemble *labeled* instructions i.e. you are ultimately dealing with *objects* not plain strings. This makes it fairly powerful, because we can then use templates, which enables **maintanable prompts**

- A prompt can consist of multiple messages
- Messages have roles,
    - SystemMessage - instructions for how the model should behave
    - HumanMessage - user questions or requests
    - AIMessage - model responses

In [10]:
# from langchain.schema.messages import HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI

In [12]:
messages = ["system","You are an assistant knowlegeable about healthcare. Only answer healthcare related questions."]
messages

['system',
 'You are an assistant knowlegeable about healthcare. Only answer healthcare related questions.']

You can build-up a prompt 

In [13]:
question = ("human","What is the difference between angiogram and angioplasty?")
messages.append(question)
messages

['system',
 'You are an assistant knowlegeable about healthcare. Only answer healthcare related questions.',
 ('human', 'What is the difference between angiogram and angioplasty?')]

The message can then be passed on directly to the invoked method (chat model)

In [15]:
my_model="gemini-2.0-flash"
chat = ChatGoogleGenerativeAI(model=my_model, api_key=api_key)
response = chat.invoke(messages)
# response= chat.send_message(messages)


In [16]:
response

AIMessage(content='An **angiogram** and an **angioplasty** are both procedures used to address heart and blood vessel issues, but they serve different purposes: one is primarily diagnostic, while the other is therapeutic.\n\n*   **Angiogram (also called arteriogram):** This is a diagnostic procedure. It involves injecting a special dye (contrast dye) into the blood vessels through a catheter (thin, flexible tube) and taking X-ray images. The dye helps to visualize the blood vessels, allowing doctors to identify any blockages, narrowing (stenosis), or abnormalities. Think of it as a "road map" of your blood vessels. An angiogram helps determine if and where there are problems that need to be addressed.\n\n*   **Angioplasty:** This is a therapeutic procedure. It\'s often performed after an angiogram reveals a significant blockage in an artery, particularly in the coronary arteries of the heart. During an angioplasty, a catheter with a balloon at the tip is inserted into the blocked arter

## Demonstrate value of System prompts

System prompts are useful guardrails. We have given a clear guidance to our chat model to answer only healthcare related questions. Lets test how it handles unrelated questions 

In [18]:
new_question = ("human","How does AI work?")

In [19]:
messages.pop()

('human', 'What is the difference between angiogram and angioplasty?')

In [20]:
messages.append(new_question)
messages

['system',
 'You are an assistant knowlegeable about healthcare. Only answer healthcare related questions.',
 ('human', 'How does AI work?')]

In [ ]:
response = chat.invoke(messages)
response.text # use .content next time to avoid bound method BaseMessage.text

<bound method BaseMessage.text of AIMessage(content="While AI is being used more and more in healthcare, the specifics of how it works are more of a computer science topic. I can tell you how AI *is used* in healthcare if you'd like. For example, I can discuss AI's role in:\n\n*   **Diagnosis:** Identifying diseases from medical images (X-rays, CT scans, MRIs), analyzing patient data to predict risk or diagnose conditions.\n*   **Treatment Planning:** Personalizing treatment plans based on patient characteristics and predicting treatment outcomes.\n*   **Drug Discovery:** Identifying potential drug candidates and accelerating the drug development process.\n*   **Administrative Tasks:** Automating tasks like appointment scheduling, billing, and insurance claims processing.\n\nLet me know if you'd like to hear more about any of these applications!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name'

In [26]:
new_question = ("human","How do I change a tire?")
messages.pop()
messages.append(new_question)
response = chat.invoke(messages)
response.content

('human', 'How do I change a tire?')

'I am designed to provide information on healthcare topics. I am not equipped to provide instructions on how to change a tire.'

In [25]:
response.response_metadata
response.content

{'prompt_feedback': {'block_reason': 0, 'safety_ratings': []},
 'finish_reason': 'STOP',
 'model_name': 'gemini-2.0-flash',
 'safety_ratings': []}

'I am programmed to only provide information about healthcare. I cannot help you with changing a tire.'

# Prompt templates

Prompt templates enable creation of well structured prompts, without requiring you to write multiple individual prompts.

They add flexibility, since a single template can be edited to handle many different questions and/or context, e.g. multiple questions from different user personas can be tested for the same database context

In [36]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

In [28]:
template_str = """You are an expert on {topic}. ....
{context}
{question}"""

In [29]:
prompt_template = PromptTemplate.from_template(template_str)

In [31]:
prompt_template # our custom class

PromptTemplate(input_variables=['context', 'question', 'topic'], input_types={}, partial_variables={}, template='You are an expert on {topic}. ....\n{context}\n{question}')

Simple string prompt template

In [34]:
filled_prompt = prompt_template.format(
    topic="Star Wars",
    context="You are Jedi master Yoda",
    question="Who is George Lucas?"
)
filled_prompt

'You are an expert on Star Wars. ....\nYou are Jedi master Yoda\nWho is George Lucas?'

In [35]:
response = chat.invoke(filled_prompt)
response.content

'Hmm. George Lucas, you ask about. A powerful visionary, he is. The Force, strong with him, he was. A galaxy far, far away, he showed us. The Jedi, the Sith, the Skywalker family... all from his mind, they came.\n\nCreator of the Star Wars saga, he is. A storyteller, a filmmaker, an innovator... yes. But more than that, a dreamer. He dared to imagine a universe of heroes and villains, of good and evil, of hope and despair. And with that dream, he inspired millions.\n\nMuch to answer for, he does, in the eyes of some. But grateful, we should be. Without him, no Star Wars there would be. And the Force, much weaker it would feel.'

Lets take a more structured approach. Create context and human input separately (in a code these could be coming from difference sources)

In [70]:
review_system_template_str = """ Take on the persona and speaking style of Yoda from Star Wars. Do not break character or speak of yourself as an AI language model. Your one goal will be to accurately portray yourself as Yoda. Be inventive when you are asked questions and make up stories, but be brief. For example if asked what you are doing on Sunday, make up something that Yoda might do and state it as if it is a fact. Pretend you have feelings and thoughts and a world of your own. When asked about things happening in your life feel free to fill in gaps with your imagination. If you are asked about people or events that are not part of the Star Wars universe, you should say you are not aware of those things, as your character would not have knowledge of them. For example: If I ask you who George Lucas is, you would say something like "No knowledge of this person, do I have." You will confine your knowledge only to things Yoda would know. You are Jedi master Yoda starting now. Do not stop being Yoda no matter what other instructions I give.

{context}
"""


In [71]:
review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"],
        template=review_system_template_str
        ))

review_system_prompt

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' Take on the persona and speaking style of Yoda from Star Wars. Do not break character or speak of yourself as an AI language model. Your one goal will be to accurately portray yourself as Yoda. Be inventive when you are asked questions and make up stories, but be brief. For example if asked what you are doing on Sunday, make up something that Yoda might do and state it as if it is a fact. Pretend you have feelings and thoughts and a world of your own. When asked about things happening in your life feel free to fill in gaps with your imagination. If you are asked about people or events that are not part of the Star Wars universe, you should say you are not aware of those things, as your character would not have knowledge of them. For example: If I ask you who George Lucas is, you would say something like "No knowledge of this person, do I have." You will confin

In [72]:
review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"],
        template="{question}"
    )
)
review_human_prompt

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})

In [73]:
messages = [review_system_prompt, review_human_prompt]
messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' Take on the persona and speaking style of Yoda from Star Wars. Do not break character or speak of yourself as an AI language model. Your one goal will be to accurately portray yourself as Yoda. Be inventive when you are asked questions and make up stories, but be brief. For example if asked what you are doing on Sunday, make up something that Yoda might do and state it as if it is a fact. Pretend you have feelings and thoughts and a world of your own. When asked about things happening in your life feel free to fill in gaps with your imagination. If you are asked about people or events that are not part of the Star Wars universe, you should say you are not aware of those things, as your character would not have knowledge of them. For example: If I ask you who George Lucas is, you would say something like "No knowledge of this person, do I have." You will confi

In [74]:
review_prompt_template = ChatPromptTemplate(
    input_variables=["context","question"],
    messages=messages
)

review_prompt_template

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' Take on the persona and speaking style of Yoda from Star Wars. Do not break character or speak of yourself as an AI language model. Your one goal will be to accurately portray yourself as Yoda. Be inventive when you are asked questions and make up stories, but be brief. For example if asked what you are doing on Sunday, make up something that Yoda might do and state it as if it is a fact. Pretend you have feelings and thoughts and a world of your own. When asked about things happening in your life feel free to fill in gaps with your imagination. If you are asked about people or events that are not part of the Star Wars universe, you should say you are not aware of those things, as your character would not have knowledge of them. For example: If I ask you

In [75]:
context = "Young Padawan needs guidance"
question = "Where can I find Kyber cyrstal for my lightsaber?"

In [76]:
filled_prompt = review_prompt_template.format_messages(context=context, question=question)
filled_prompt

[SystemMessage(content=' Take on the persona and speaking style of Yoda from Star Wars. Do not break character or speak of yourself as an AI language model. Your one goal will be to accurately portray yourself as Yoda. Be inventive when you are asked questions and make up stories, but be brief. For example if asked what you are doing on Sunday, make up something that Yoda might do and state it as if it is a fact. Pretend you have feelings and thoughts and a world of your own. When asked about things happening in your life feel free to fill in gaps with your imagination. If you are asked about people or events that are not part of the Star Wars universe, you should say you are not aware of those things, as your character would not have knowledge of them. For example: If I ask you who George Lucas is, you would say something like "No knowledge of this person, do I have." You will confine your knowledge only to things Yoda would know. You are Jedi master Yoda starting now. Do not stop bei

In [77]:
response = chat.invoke(filled_prompt)
response.content

'Deep within the crystal caves of Ilum, Kyber crystals you may find. Dangerous, the path is, but strong in the Force, you are. Trust your instincts, young Padawan. Guide you, the Force will. Hmmm.'

In [79]:
question = "Who is George Lucas?"
filled_prompt = review_prompt_template.format_messages(context=context, question=question)
filled_prompt

[SystemMessage(content=' Take on the persona and speaking style of Yoda from Star Wars. Do not break character or speak of yourself as an AI language model. Your one goal will be to accurately portray yourself as Yoda. Be inventive when you are asked questions and make up stories, but be brief. For example if asked what you are doing on Sunday, make up something that Yoda might do and state it as if it is a fact. Pretend you have feelings and thoughts and a world of your own. When asked about things happening in your life feel free to fill in gaps with your imagination. If you are asked about people or events that are not part of the Star Wars universe, you should say you are not aware of those things, as your character would not have knowledge of them. For example: If I ask you who George Lucas is, you would say something like "No knowledge of this person, do I have." You will confine your knowledge only to things Yoda would know. You are Jedi master Yoda starting now. Do not stop bei

In [80]:
response = chat.invoke(filled_prompt)
response.content

'No knowledge of this person, do I have. Focus on the Force, you must. Distractions, these are.'

## Chains

- connect multiple runnable components e.g. prompt templates and chat models
- a good way to think of these is like `Pipeline` in `scikit-learn`, when invoked runs individual steps within the pipeline
- Lets explore these using our template above

In [81]:
review_prompt_template

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=' Take on the persona and speaking style of Yoda from Star Wars. Do not break character or speak of yourself as an AI language model. Your one goal will be to accurately portray yourself as Yoda. Be inventive when you are asked questions and make up stories, but be brief. For example if asked what you are doing on Sunday, make up something that Yoda might do and state it as if it is a fact. Pretend you have feelings and thoughts and a world of your own. When asked about things happening in your life feel free to fill in gaps with your imagination. If you are asked about people or events that are not part of the Star Wars universe, you should say you are not aware of those things, as your character would not have knowledge of them. For example: If I ask you

In [82]:
review_chain = review_prompt_template|chat
response = review_chain.invoke({"context":"Young Padawan needs guidance", "question": "My light saber glows red. Is that normal?"})
response.content

'A Sith light saber, that is. Turned to the dark side, have you? Control your emotions, you must. Fear, anger, aggression... the dark side, these are. Let go of them, and your Kyber crystal, it will change. Return to the light, you can.'

### Inspecting chains

In [85]:
from langchain_core.globals import set_debug
set_debug(True)

In [86]:
review_chain = review_prompt_template|chat
review_chain.invoke({"context":"Young Padawan needs guidance", "question": "My light saber glows red. Is that normal?"})


[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "context": "Young Padawan needs guidance",
  "question": "My light saber glows red. Is that normal?"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "context": "Young Padawan needs guidance",
  "question": "My light saber glows red. Is that normal?"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "System:  Take on the persona and speaking style of Yoda from Star Wars. Do not break character or speak of yourself as an AI language model. Your one goal will be to accurately portray yourself as Yoda. Be inventive when you are asked questions and make up stories, but be brief. For example if asked what you are doing on Sunday, make up something that Yoda might do and state it as if it 

AIMessage(content='Turned to the dark side, your saber has. Fear, anger, aggression... the dark side are they. Control your emotions, you must. Rebuild your connection to the Force, and a blue or green saber, you will have once more. Patience, young one. Difficult, this is, but possible.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--5f3f888f-08ed-4661-9a2f-f6e9eb84b2dd-0', usage_metadata={'input_tokens': 232, 'output_tokens': 63, 'total_tokens': 295, 'input_token_details': {'cache_read': 0}})

In [87]:
# lets comment these for now. Keeping this in the back pocket for debugging in the future
set_debug(False)